In [1]:
from pathlib import Path
import re

from pysam import FastaFile
import pandas as pd

In [2]:
DATA_ROOT = Path('/Users/liang/Box/MyCPTAC/CPTAC_proteome_v3.0/')

refseq_fasta_pth = DATA_ROOT / 'DCC/RefSeq_20111201/20111201_RefSeq_Human_37-Mouse_37_Trypsin.renamed.fasta.gz'

phospho_pth = DATA_ROOT / 'CPTAC2/TCGA_OV_retrospective/mmc3_phosphoproteome_ischemia_filtered_peptide_annotation.tsv.gz'

In [3]:
protein_fa = FastaFile(str(refseq_fasta_pth))

In [4]:
def calc_peptide_start_end(peptide, protein_id, sites):
    # Make sure the peptide is part of the protein sequence
    protein_seq = protein_fa.fetch(protein_id)
    if not peptide.upper() in protein_seq:
        return [None, None, False, 'Peptide not found in protein']
    
    first_res = sites[0][0].lower()
    first_res_loc = int(sites[0][1:])
    first_res_ix = peptide.find(first_res) 
    
    peptide_start = first_res_loc - first_res_ix
    peptide_end = first_res_loc + len(peptide) - first_res_ix - 1
    if protein_seq[peptide_start - 1:peptide_end] != peptide.upper():
        return [None, None, False, 'Invalid location']
    return [peptide_start, peptide_end, True, None]

In [5]:
peptide_df = pd.read_table(phospho_pth)
peptide_df.columns

Index(['refseq_prot_id', 'Peptide', 'Site'], dtype='object')

In [6]:
peptide_df = peptide_df.rename(columns={'Peptide': 'original_peptide'})
peptide_df['phosphosites'] = peptide_df['Site'].str.split('-', 1, expand=True).iloc[:, 1].str.findall(r'[A-Z]\d+[a-z]')
peptide_df['peptide'] = peptide_df['original_peptide'].str.slice(start=2, stop=-2).str.replace(r'([STY])\*', lambda m: m.group(1).lower())
peptide_df.head(10)

,refseq_prot_id,original_peptide,Site,phosphosites,peptide
0,NP_000022.3,K.SS*PAFGDRR.C,ALAD-S215s,[S215s],SsPAFGDRR
1,NP_000022.3,R.DAAKS*SPAFGDRR.C,ALAD-S214s,[S214s],DAAKsSPAFGDRR
2,NP_000022.3,R.DAAKSS*PAFGDR.R,ALAD-S215s,[S215s],DAAKSsPAFGDR
3,NP_000022.3,R.DAAKSS*PAFGDRR.C,ALAD-S215s,[S215s],DAAKSsPAFGDRR
4,NP_000028.3,-.PY*SVGFR.E,ANK1-Y3y,[Y3y],PySVGFR
5,NP_000028.3,-.PYS*VGFR.E,ANK1-S4s,[S4s],PYsVGFR
6,NP_000028.3,K.NGY*TPLHIAAK.K,ANK1-Y603y,[Y603y],NGyTPLHIAAK
7,NP_000028.3,K.NGYT*PLHIAAK.K,ANK1-T604t,[T604t],NGYtPLHIAAK
8,NP_000028.3,K.RLGYIS*VTDVLK.V,ANK1-S781s,[S781s],RLGYIsVTDVLK
9,NP_000028.3,R.IT*HSPTVSQVTER.S,ANK1-T1684t,[T1684t],ItHSPTVSQVTER


Check what residues are modified

In [7]:
peptide_df['phosphosites'].explode().apply(lambda x: x[0]).value_counts()

S    5875
T    1572
Y     465
Name: phosphosites, dtype: int64

In [8]:
# Keep only STY that are phosphosites
peptide_df['phosphosites'] = peptide_df['phosphosites'].apply(
    lambda l: [x[:-1] for x in l if x[0] in 'STY']
)

In [9]:
# Calculate the peptide location
peptide_loc_range_df = pd.DataFrame(
    map(calc_peptide_start_end, 
        peptide_df.peptide, peptide_df.refseq_prot_id, peptide_df.phosphosites),
    index=peptide_df.index,
    columns=['peptide_start', 'peptide_end', 'peptide_loc_valid', 'peptide_loc_invalid_reason'],
).astype({
    'peptide_start': pd.Int64Dtype(),
    'peptide_end': pd.Int64Dtype(),
})

In [10]:
# Combine the location and peptide information
peptide_with_loc_df = pd.concat([peptide_df, peptide_loc_range_df], axis=1)
# List all the invalid peptides
peptide_with_loc_df[~peptide_loc_range_df['peptide_loc_valid']]

,refseq_prot_id,original_peptide,Site,phosphosites,peptide,peptide_start,peptide_end,peptide_loc_valid,peptide_loc_invalid_reason
118,NP_000319.1,K.RVPS*INQK.I,RPGR-S961s,[S961],RVPsINQK,<NA>,<NA>,False,Invalid location
136,NP_000356.1,K.TAT*PQQAQEVHEK.L,TPI1-T215t,[T215],TAtPQQAQEVHEK,<NA>,<NA>,False,Invalid location
137,NP_000356.1,R.IIY*GGSVTGATCK.E,TPI1-Y246y,[Y246],IIyGGSVTGATCK,<NA>,<NA>,False,Invalid location
138,NP_000356.1,R.IIYGGS*VTGATCK.E,TPI1-S249s,[S249],IIYGGsVTGATCK,<NA>,<NA>,False,Invalid location
139,NP_000356.1,R.KQS*LGELIGTLNAAK.V,TPI1-S58s,[S58],KQsLGELIGTLNAAK,<NA>,<NA>,False,Invalid location
...,...,...,...,...,...,...,...,...,...
7764,NP_997657.1,K.SPPREGS*QGELTPANSQSR.M,GPS1-S474s,[S474],SPPREGsQGELTPANSQSR,<NA>,<NA>,False,Invalid location
7765,NP_997657.1,K.SPPREGSQGELT*PANSQSR.M,GPS1-T479t,[T479],SPPREGSQGELtPANSQSR,<NA>,<NA>,False,Invalid location
7766,NP_997657.1,R.GERDS*QTQAILTK.L,GPS1-S240s,[S240],GERDsQTQAILTK,<NA>,<NA>,False,Invalid location
7767,NP_997657.1,R.GERDSQT*QAILTK.L,GPS1-T242t,[T242],GERDSQtQAILTK,<NA>,<NA>,False,Invalid location


In [11]:
out_df = peptide_with_loc_df.loc[
    :,
    ['refseq_prot_id', 'original_peptide', 
     'peptide', 'phosphosites', 
     'peptide_start', 'peptide_end', 'peptide_loc_valid', 'peptide_loc_invalid_reason']
]
out_df['phosphosites'] = out_df['phosphosites'].str.join(';')
out_df.to_csv(
    Path(phospho_pth.parent, phospho_pth.stem).with_suffix('.peptide_location.tsv.gz'),
    sep='\t',
    index=False
)

In [12]:
out_df.head()

,refseq_prot_id,original_peptide,peptide,phosphosites,peptide_start,peptide_end,peptide_loc_valid,peptide_loc_invalid_reason
0,NP_000022.3,K.SS*PAFGDRR.C,SsPAFGDRR,S215,214,222,True,None
1,NP_000022.3,R.DAAKS*SPAFGDRR.C,DAAKsSPAFGDRR,S214,210,222,True,None
2,NP_000022.3,R.DAAKSS*PAFGDR.R,DAAKSsPAFGDR,S215,210,221,True,None
3,NP_000022.3,R.DAAKSS*PAFGDRR.C,DAAKSsPAFGDRR,S215,210,222,True,None
4,NP_000028.3,-.PY*SVGFR.E,PySVGFR,Y3,2,8,True,None
